In [1]:
import numpy as np
import pv_4_5
from pydae.utils import read_data, save_json

In [2]:
M = 4
N = 5
model = pv_4_5.model()
model.ini({},'xy_0.json')
# 'bs_MV0102_MV0101':0.0

True

In [3]:
measurements = {}
conditions = {}
grid_data = read_data('pv_4_5.json')

buse_names = [item['name'] for item in grid_data['buses']]

# Generators active and reactive power (W and var)
# Low Voltage side
for i_m in range(1,M+1):
    for i_n in range(1,N+1):
        name = 'LV' + f"{i_m}".zfill(2) + f"{i_n}".zfill(2)

        S_base = model.get_value(f'S_n_{name}')
        U_base = grid_data['buses'][buse_names.index(name)]['U_kV']*1000

        P = model.get_value(f'p_s_{name}')*S_base
        Q = model.get_value(f'q_s_{name}')*S_base
        U_m = model.get_value(f'V_{name}')*U_base

        S = P + 1j*Q
        S_m = np.abs(S)
        I_m = S_m/(np.sqrt(3)*U_m)


        measurements.update({f'P_{name}':P})
        measurements.update({f'Q_{name}':Q})
        measurements.update({f'U_{name}':U_m}) # "m" means "module"
        measurements.update({f'I_{name}':I_m})    

        
        conditions.update({f'irrad_{name}':model.get_value(f'irrad_{name}')})   


# Feeders
i_n = 1
for i_m in range(1,M+1):
    name = 'POI_MV_MV' + f"{i_m}".zfill(2) + f"{i_n}".zfill(2) 
 
    P_Feeder_pu,Q_Feeder_pu = model.get_mvalue([f'p_line_{name}',f'q_line_{name}'])
    P =-P_Feeder_pu*grid_data['system']['S_base']
    Q =-Q_Feeder_pu*grid_data['system']['S_base']
    S_base = model.get_value(f'S_n_{name}')
    U_base = grid_data['buses'][buse_names.index('POI_MV')]['U_kV']*1000

    S = P + 1j*Q
    S_m = np.abs(S)
    U_m = model.get_value(f'V_POI_MV')*U_base
    I_m = S_m/(np.sqrt(3)*U_m)


    measurements.update({f'P_{name}':P})
    measurements.update({f'Q_{name}':Q})
    measurements.update({f'U_{name}':U_m}) # "m" means "module"
    measurements.update({f'I_{name}':I_m})  


# Powers and voltages at POI
P_POI_pu,Q_POI_pu = model.get_mvalue([f'p_line_POI_GRID',f'q_line_POI_GRID'])
P = P_POI_pu*grid_data['system']['S_base']
Q = Q_POI_pu*grid_data['system']['S_base']
S_base = model.get_value(f'S_n_{name}')
U_base = grid_data['buses'][buse_names.index('POI')]['U_kV']*1000

S = P + 1j*Q
S_m = np.abs(S)
I_m = S_m/(np.sqrt(3)*U_m)


measurements.update({f'P_POI':P})
measurements.update({f'Q_POI':Q})
measurements.update({f'U_POI':U_m}) # "m" means "module"
measurements.update({f'I_POI':I_m})  

save_json(measurements,file='measurements.json')
save_json(conditions,file='conditions.json')

In [4]:
name

'POI_MV_MV0401'

In [5]:
measurements

{'P_LV0101': 749825.0,
 'Q_LV0101': -7.976988751349557e-11,
 'U_LV0101': 402.4901677775121,
 'I_LV0101': 1075.5832073751808,
 'P_LV0102': 749825.0000000001,
 'Q_LV0102': 1.7712790224935563e-09,
 'U_LV0102': 404.9022868685796,
 'I_LV0102': 1069.1756496194419,
 'P_LV0103': 749824.9999999997,
 'Q_LV0103': 1.7270624804249954e-09,
 'U_LV0103': 407.29570269526107,
 'I_LV0103': 1062.8927895146878,
 'P_LV0104': 749824.9999999998,
 'Q_LV0104': -1.2028689343370752e-09,
 'U_LV0104': 409.67030962249834,
 'I_LV0104': 1056.731853460479,
 'P_LV0105': 749824.9999999997,
 'Q_LV0105': -3.284993310672963e-10,
 'U_LV0105': 412.02600667641053,
 'I_LV0105': 1050.6901471758388,
 'P_LV0201': 749825.0000000001,
 'Q_LV0201': 2.642298798787544e-10,
 'U_LV0201': 402.4901677775123,
 'I_LV0201': 1075.5832073751806,
 'P_LV0202': 749825.0000000002,
 'Q_LV0202': 3.982466418178248e-09,
 'U_LV0202': 404.9022868685798,
 'I_LV0202': 1069.1756496194416,
 'P_LV0203': 749824.9999999995,
 'Q_LV0203': -2.2702639102882412e-09,


In [6]:
model.get_value('bs_MV0102_MV0101')

0.00048

In [7]:
model.report_z()

V_POI_MV =  0.99
V_POI =  1.00
V_GRID =  1.00
V_BESS =  0.99
V_LV0101 =  1.01
V_MV0101 =  1.00
V_LV0102 =  1.01
V_MV0102 =  1.01
V_LV0103 =  1.02
V_MV0103 =  1.01
V_LV0104 =  1.02
V_MV0104 =  1.02
V_LV0105 =  1.03
V_MV0105 =  1.02
V_LV0201 =  1.01
V_MV0201 =  1.00
V_LV0202 =  1.01
V_MV0202 =  1.01
V_LV0203 =  1.02
V_MV0203 =  1.01
V_LV0204 =  1.02
V_MV0204 =  1.02
V_LV0205 =  1.03
V_MV0205 =  1.02
V_LV0301 =  1.01
V_MV0301 =  1.00
V_LV0302 =  1.01
V_MV0302 =  1.01
V_LV0303 =  1.02
V_MV0303 =  1.01
V_LV0304 =  1.02
V_MV0304 =  1.02
V_LV0305 =  1.03
V_MV0305 =  1.02
V_LV0401 =  1.01
V_MV0401 =  1.00
V_LV0402 =  1.01
V_MV0402 =  1.01
V_LV0403 =  1.02
V_MV0403 =  1.01
V_LV0404 =  1.02
V_MV0404 =  1.02
V_LV0405 =  1.03
V_MV0405 =  1.02
p_line_POI_GRID =  0.15
q_line_POI_GRID = -0.01
p_line_GRID_POI = -0.15
q_line_GRID_POI =  0.02
p_line_BESS_POI_MV = -0.00
q_line_BESS_POI_MV =  0.00
p_line_POI_MV_BESS =  0.00
q_line_POI_MV_BESS =  0.00
p_line_MV0101_POI_MV =  0.04
q_line_MV0101_POI_MV = -0.

In [8]:
buse_names

['POI_MV',
 'POI',
 'GRID',
 'BESS',
 'LV0101',
 'MV0101',
 'LV0102',
 'MV0102',
 'LV0103',
 'MV0103',
 'LV0104',
 'MV0104',
 'LV0105',
 'MV0105',
 'LV0201',
 'MV0201',
 'LV0202',
 'MV0202',
 'LV0203',
 'MV0203',
 'LV0204',
 'MV0204',
 'LV0205',
 'MV0205',
 'LV0301',
 'MV0301',
 'LV0302',
 'MV0302',
 'LV0303',
 'MV0303',
 'LV0304',
 'MV0304',
 'LV0305',
 'MV0305',
 'LV0401',
 'MV0401',
 'LV0402',
 'MV0402',
 'LV0403',
 'MV0403',
 'LV0404',
 'MV0404',
 'LV0405',
 'MV0405']

In [9]:
Efficiency = 0.15
Irradiance = 1000 # W/m**2
P_n = 1e6

Area_per_PV = P_n/(Irradiance*Efficiency) 
Side_per_PV = np.sqrt(Area_per_PV)
Side_per_PV

81.64965809277261

In [10]:
120*7*20e3*np.sqrt(3)/1e6

29.098453567157136